In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image
from os import listdir

In [11]:
import glob
import PIL

In [31]:
parasite_files = glob.glob('Parasitized/*.png')

In [36]:
uninfected_files = glob.glob('Uninfected/*.png')

In [33]:
X_parasite = []
Y_parasite = []

In [35]:
for file in parasite_files:
    image_data = image.imread(file)
    X_parasite.append(image_data)
    Y_parasite.append(1)
    # append 1 to Y as target infected variable

In [38]:
X_uninfected = []
Y_uninfected = []

In [ ]:
for file in uninfected_files:
    image_data2 = image.imread(file)
    X_uninfected.append(image_data2)
    Y_uninfected.append(0)
    # appent 0 to Y as target not infected